# AlphaGo Zero for connect 2

https://web.stanford.edu/~surag/posts/alphazero.html <br>
https://www.youtube.com/watch?v=62nq4Zsn8vc&ab_channel=JoshVarty <br>
https://github.com/JoshVarty/AlphaZeroSimple <br>

### Value network and self play
1) Play a bunch of games against yourself (using policy network to decide where to move) and record whether a given board state eventually lead to a win or a loss. We take a snapshot of what the board looks like when presented to each player and want to determine how good of a position they are currently in before they take a move. <br>

`
 BOARD STATE 
[ 0  0  0  0] # Start of the game, presented to Player 1. Player 1 plays in the first position.
[-1  0  0  0] # Player 2 is presented with this board. Player 2 plays in the third position.
[ 1  0 -1  0] # Player 1 is presented with this board. Player 1 plays in the second position
[-1 -1  1  0] # Player 2 is presented with this board. Player 2 was lost as Player 1 can connect 2
`

2) Create a labelled training set for the value NN by going back through all of the game states held by Player 1 and marking them with a reward of 1. Similarily go through all of the states held by Player 2 and mark them with a reward of -1. <br>

` 
  BOARD STATE   RES 
([ 0  0  0  0], 1) # Player 1 
([-1  0  0  0],-1) # Player 2 
([ 1  0 -1  0], 1) # Player 1 
([-1 -1  1  0],-1) # Player 2 
`

3) After completing thousands of self play games, we shuffle up the board training data and feed it into our neural network to train it to recognise what the probability of winning is given a particular board state. The output will be from -1 to 1. This is essentially an image recognition task, which is why AlphaGo uses resnets.

<img src="value_network.PNG" alt="drawing" width="400" align='left'/> 
<br><br><br><br><br><br><br><br><br><br>


### Policy Network
Takes a game state / board as input and outputs next move a probability distribution of next moves, with move with the highest probability being the move which the policy network thinks will do the best.

<img src="policy_network.PNG" alt="drawing" width="400" align='left'/> 
<br><br><br><br><br><br><br><br>

For example, based upon the output above, the network thinkgs we will win 88% of the time if you make a move into the first position and have a 10% probability of winning in the last position. Wierdly, it outputs a non-zero chance of winning if we take invalid moves where there are already pieces. This is because we cannot control the NN to output 0 values for invalid moves. Instead, we will have to write code to mask out invalid moves and redistribute the values.

The way we train the network is to actually encourage it to give the same output as the Monte Carlo Tree search (more on that later, it actually uses the relative count of how often a particular state gets visited when conducting MCTS). To do this we create a dataset that includes a board state, and also what the MCTS suggested we do at that position. <br>

`
BOARD STATE             MCTS        
([ 0  0  0  0], [0.1, 0.4, 0.4, 0.1]) 
([-1  0  0  0], [0.0, 0.3, 0.3, 0.3]) 
([ 1  0 -1  0], [0.0, 0.8, 0.0, 0.2]) 
`

There is actually an interesting circular dependency, where by we are training our policy network to output results that match the MCTS, but we actually use our policy to network to choose the most promising next positions to expolore with MCTS. In theory, improving the policy network will improve the MCTS and visa versa. <br>


### AlphaGo Zero actually uses the same network to output the policy and value

<img src="policy_value.PNG" alt="drawing" width="400" align='left'/>
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

### MCTS
Normally MCTS would entail doing rollouts until the a particular game is ran to conclusion and then the result is backed up to update the previous states; however, with AlphaGo Zero we actually use the results from the value network to score each state (called "virtual losses"). Of course, if we do happen to reach a terminal game state, then the actual result will be used instead and backed up. The lack of focus of complete rollouts can be hugely befefitial in a game like Go where there are any moves until termination.

How it works: <br>
1) We feed the root node (starting game state) `[0 0 0 0]` into our combined value/policy network at get its value and next move probabilities. Given a randomly initialised NN to start with, lets say it outputs: <br>
`0` for our value which remember is between -1, 1. <br>
`[0.25 0.25 0.25 0.25]` for the action probabilities. <br>

2) We calculate a score called **UCB (upper confidence bound)** for each child node, and pick the one with the highest. The UCB takes into account: <br>
- Child node visit counts vs other children at this level.
- Policy network probabilities for child state.
- Value network values for child state.

For our first action / step the UCB components are:
- The visit counts of each child node (next action) will clearly be zero.
- Action probabilites given by our randomly initialised NN were equal for each child. 
- Because we havn't visited any child nodes yet, we don't have a value network score for them, so the values are assumed to be 0 (remember, its score from -1, 1).

<ins>Upper Confidence Bound (UCB)<ins><br>
    
<img src="UCB.PNG" alt="drawing" width="550" align='left'/>
<br><br><br><br><br><br><br><br>

UCB score for each child node: <br>
UCB(s,a) = -0 + 0.25 ∗ sqrt(0) / (1 + 0) = **0** <br>

3) Given all child nodes have an equal UCB score, we pick one at random (the first one). <br>
We then evaluate the state `[-1 0 0 0]` (from the perspective of player 2 now) by passing it to the NN and increasing the `visit count` of this node (action) by one. <br>
**The NN outputs:** <br>
`value: 0.5` <br>
`proabilities: [0.33 0.33 0.33]` for the three remaining valid moves. <br>

<img src="nodes1.PNG" alt="drawing" width="800" align='left'/> 
<br><br><br><br><br><br><br><br><br><br><br>

4) We `backup` the `0.5` value of this child node to the root node, where the value of the root becomes the **average** of its visited child nodes, but its **negative**, because its from the perspective of the other player, so the root node value becomes `-0.5`. <br>

5) We calculate the UCB score for each 1st level state to decide what to do next: <br>
Node1 UCB = -0.5 + 0.25 * sqrt(1) / 2 = -0.5 + 0.125 = **-0.375** <br>
Node2,3,4 UCB = 0 + 0 * sqrt(1) / 0 = **0** <br>

Given that the first node we visited didnt give us a positive value (with respect to the root/parent node) it makes sense for us to explore other nodes. In this example we set our NN to ouput the same value for each state (because it's randomly initialized and not trained yet). <br>
<img src="nodes2.PNG" alt="drawing" width="800" align='left'/> 
<br><br><br><br><br><br><br>

6) This leads us to take another random action (the first node). Because this actions board state has already been evaluated by our NN, we expand its node and look for the next possible moves. This leads to three valid moves we could take. <br>
Given that none of them been visited yet, they have a value of 0, equal prior probabilities and therefore equal UCB scores. We then pick the one at random (the first one) and evaluate it with our NN.<br>
We evaluate the state `[1 -1 0 0]` (from the perspective of player 2 now) by passing it to the NN and increasing the `visit count` of this node (action) by one. <br>
**The NN outputs:** <br>
`value: 0.5` <br>
`proabilities: [0.5 0.5]` for the two remaining valid moves. <br>

<img src="nodes.PNG" alt="drawing" width="800" align='left'/> 
<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

7) We `backup` the `0.5` value of this child node to its parent node and ultimately the root node (remebering to alternative negative and positive to account for the perspective of each player). <br>
*It is interesting to note that the initial `value` that the NN outputted for a state is used as part of the average for a states final `value`. e.g the state at `[-1 0 0 0]` here gets an averaged value of 0 becase its initial NN value was 0.5, and now its child got a value of -0.5 (from the parents perspective), leaving an average of 0.  <br>

8) We repeat the above steps 5-7 for the set number of MCTS simulations (100 as default). We then use the visit counts for the next actions (1st level) to create a probability distrbution. Lets say we have the following visit counts `[60 20 10 10]` we noramlize it to `[0.6 0.2 0.2 0.1]`. <br>
This distribution is used to take the best action for our **first move during self play** and also to collected to train the policy NN to output probabilities more inline with MCTS. 

9) We repeat steps all of the steps 1-8 but this time inputting the new state from self play. This continues until we reach the end of a game of self play (`episode`) and use the actual result to backpropegate label all of the self play states (more info at the start of this doc).

10) We play a set number of full self play games / `episodes` (default is 100) and then train the NN. We use the collected value labels and MCTS visit probabilites for each self play state to train the value/policy NN. Both are used as part of the loss function to train both parts of the NN. We pass the shuffled dataset to the NN for a set number of `epochs` (default is 2).

11) We repeat the above steps for a number of `training iterations` (default is 500), meaning that the agent will have played 500 * 100 = 5,000 self play games. Each game takes 4 moves to complete, with each move needing 100 MCTS simulations, so that is 500 * 4 * 100 = 20,000 total MCTS simulations.

In [1]:
import numpy as np
import pandas as pd
import logging
import sys
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import backend as K

In [2]:
logging.basicConfig(stream=sys.stdout, format='%(levelname)-8s %(message)s')
log = logging.getLogger(__name__)

#log.setLevel(logging.DEBUG)
log.setLevel(logging.INFO)

## Connect 2 game

In [27]:
def test_nn(board):
    ''' Feed state into neural network and get value and probability array'''
    state_value = 0.5
    child_probs = np.array([0.25, 0.25, 0.25, 0.25])
    return state_value, child_probs


def get_valid_moves(board):
    mask = board == np.array([0,0,0,0])
    valid_moves = np.nonzero(mask)[0]
    return valid_moves


def clean_nn_probs(board, child_probs):
    '''Mask out invalid moves from NN probabilities array'''
    mask = board == np.array([0,0,0,0])
    masked_probs = child_probs * mask
    rescale_probs = masked_probs / np.sum(masked_probs)
    return rescale_probs


class Node:
    '''Create state nodes'''
    def __init__(self, player, state=None, prior_prob=0):
        self.player = player
        self.state = state.copy()
        self.prior_prob = prior_prob
        self.total_value = 0
        self.value = 0
        self.visit_count = 0
        self.ucb = 0
        self.children = {}
     
    def create_children(self, child_probs, children):
        for action in children:
            child_state = self.state.copy()
            child_state[action] = 1
            flip_state = flip_board(child_state)
            self.children[action] = Node(prior_prob=child_probs[action],
                                         state=flip_state,
                                         player=-self.player)
        self.update_child_ucbs()
                
    def update_child_ucbs(self):
        '''Calculate ucb score for each child state'''
        scores = []
        for action, child in self.children.items():
            child.ucb = (-child.value + child.prior_prob
                        * (np.sqrt(self.visit_count) / (1 + child.visit_count)))
            scores.append((child.ucb, action, -child.value, child.prior_prob,
                           self.visit_count, child.visit_count))
            
        sorted_scores = sorted(scores, reverse=True)
        log.debug(f'child scores: {sorted_scores}')
        self.best_child = sorted_scores[0][1]

    
    def update_value(self, value):
        self.total_value += value
        self.visit_count += 1
        self.value = self.total_value / self.visit_count

            
    def backup(self, search_path, reward):
        reward_perspective = -1
        for node in reversed(search_path):
            node.update_value(reward * reward_perspective)
            node.update_child_ucbs()
            reward_perspective *= -1
   

def take_move(board, position):
    assert board[position] == 0 , 'invalid move, place already taken'
    board[position] = 1
    return board


def flip_board(board):
    board = board.copy() * -1
    return board


def is_loss(board):
    total = 0
    for board_marker in board:
        if board_marker in [0, 1]:
            total = 0
        else:
            total += board_marker
        if total == -2:
            return True
    return False 
  
    
def end_game(board, valid_moves):
    if is_loss(board): # loss
        reward = -1
        log.debug('loss')
    elif len(valid_moves) == 0: # draw
        reward = 0
        log.debug('draw')
    else:
        reward = None
    return reward
        

def MCTS(board, cur_player, num_simulations=100):
    root = Node(player=cur_player, state=board)

    for _ in range(num_simulations + 1):
        log.debug(f'simulation no.: {_}')
        node = root
        search_path = [node]
        reward = 0

        while len(node.children) > 0:
            log.debug('expand children')
            node = node.children[node.best_child]
            board = node.state
            log.debug(f'board: {board}')
            cur_player = node.player
            search_path.append(node)

        valid_moves = get_valid_moves(board)
        log.debug(f'cur player: {cur_player}')
        log.debug(f'board: {board}')
        log.debug(f'valid moves: {valid_moves}')
        reward = end_game(board, valid_moves)
        if reward is None: # can move
            state_value, child_probs = test_nn(board)
            reward = state_value
            clean_child_probs = clean_nn_probs(board, child_probs)
            node.create_children(child_probs=clean_child_probs, children=valid_moves)

        log.debug(f'reward: {reward}')
        node.update_value(reward)
        node.backup(search_path[:-1], reward)
        log.debug('------------')
        
    return root


def calc_mcts_probs(root):
    board_template = np.array([0,0,0,0])
    for action, node in root.children.items():
        board_template[action] = node.visit_count
    mcts_probs = board_template / board_template.sum()
    return mcts_probs


def select_temperature_based_action(root, temperature):
    visit_counts = np.array([child.visit_count for child in root.children.values()])
    actions = [action for action in root.children.keys()]
    if temperature == 0:
        action = actions[np.argmax(visit_counts)]
    elif temperature == float("inf"):
        action = np.random.choice(actions)
    else:
        # See paper appendix Data Generation
        visit_count_distribution = visit_counts ** (1 / temperature)
        visit_count_distribution = visit_count_distribution / sum(visit_count_distribution)
        action = np.random.choice(actions, p=visit_count_distribution)
    return action
    
    
def episode(num_mcts_simulations=1500, temperature=0.1, board=np.array([0,0,0,0])):
    cur_player = 1
    reward = None
    train_examples = []
    board = board.copy()
    while True:
        root = MCTS(board, cur_player=cur_player, num_simulations=num_mcts_simulations)
        valid_moves = get_valid_moves(board)
        reward = end_game(board, valid_moves)
        if reward is None:
            mcts_probs = calc_mcts_probs(root)
            train_examples.append([board.copy(), mcts_probs.copy()])
            action = select_temperature_based_action(root, temperature=temperature)
            board = take_move(board, position=action)
            board = flip_board(board)
            cur_player *= -1
        else:
            train_examples.append([board, np.array([0,0,0,0])])
            return train_examples, reward
        

def sort_train_data(train_examples, reward):
    X = np.vstack([board for board, probs in train_examples])
    policy_y = np.vstack([probs for board, probs in train_examples])
    value_y = []
    for _ in range(len(train_examples)):
        value_y.append(reward)
        reward *= -1
    value_y = np.vstack(list(reversed(value_y)))
    return X, policy_y, value_y


class Network:
    def __init__(self, lr):
        self.policy_model, self.value_model = self.build_network(lr=lr)
    
    
    def build_network(self, board_size=4, lr=1e-4):
        inputs = Input(shape=(board_size))
        dense_1 = Dense(units=100, activation='relu')(inputs)
        dense_2 = Dense(units=100, activation='relu')(dense_1)
        dense_3 = Dense(units=100, activation='relu')(dense_2)
        policy = Dense(units=board_size, activation='softmax')(dense_3)
        value = Dense(units=1, activation=None)(dense_3)
         

        def custom_loss(y_true, y_pred):
            return categorical_crossentropy(y_true, y_pred, from_logits=False)
        
        policy_model = tf.keras.Model(inputs=inputs, outputs=policy)
        policy_model.compile(optimizer=Adam(lr=lr), loss=custom_loss)#, run_eagerly=True) #debug
        value_model = tf.keras.Model(inputs=inputs, outputs=value)
        value_model.compile(optimizer=Adam(lr=lr), loss='mse')
        return policy_model, value_model
    
    
    def learn(self, X, policy_y, value_y, epochs=1, loops=10, verbose=0):
        for _ in range(loops):
            self.policy_model.fit(X, policy_y, shuffle=False, epochs=epochs, verbose=verbose)
            self.value_model.fit(X, value_y, shuffle=False, epochs=epochs, verbose=verbose)

In [59]:
#############
train = True
#############

if train:
    num_iterations = 5
    num_episodes = 100
    mcts_simulations = 1500

    alpha_go = Network(lr=1e-5)
    for _ in tqdm(range(num_iterations)):
        X, policy_y, value_y = [], [], []
        for _ in range(num_episodes):
            train_examples, reward = episode(num_mcts_simulations=mcts_simulations,
                                             temperature=10)
            ep_X, ep_policy_y, ep_value_y = sort_train_data(train_examples, reward)
            X.extend(ep_X)
            policy_y.extend(ep_policy_y)
            value_y.extend(ep_value_y)
        #print(pd.DataFrame(np.concatenate([X, policy_y, value_y], axis=1)))
        alpha_go.learn(np.vstack(X), np.vstack(policy_y), np.vstack(value_y),
                       epochs=1, loops=200, verbose=0)
    alpha_go.policy_model.save_weights('alpha_go_policy_weights/')  
    alpha_go.value_model.save_weights('alpha_go_value_weights/')
else:
    print('loading pre-trained weights')
    alpha_go = Network(lr=1e-5)
    alpha_go.policy_model.load_weights('alpha_go_policy_weights/')
    alpha_go.value_model.load_weights('alpha_go_value_weights/')


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:01<00:00, 132.21s/it]


In [62]:
# board = np.array([[0,-1,1,0]])
# alpha_go.policy_model.predict(board), alpha_go.value_model.predict(board)

In [63]:
# pd.options.display.max_rows = 1000
# data = pd.DataFrame(np.concatenate([X, policy_y, value_y], axis=1))
# data = data[(
#     (data[0] == board[0][0])
#   & (data[1] == board[0][1])
#   & (data[2] == board[0][2])
#   & (data[3] == board[0][3]) 
# )]
# data

# Play against alpha go

In [66]:
while True:
    while True:
        difficulty = input('Please select difficulty [(easy/hard]')
        if difficulty not in ['easy', 'hard', 'e', 'h']:
            print('Sorry, I do not understand what you just typed, please try again')
        else:
            break
            
    while True:
        player_start = input('Would you like to start? [(y/n]')
        if player_start not in ['y','n']:
            print('Sorry, I do not understand what you just typed, please try again')
        else:
            break
        
    board = np.array([0,0,0,0])
    print('--------------------------------------------------------')
    print('Welcome to a new game of connect 2 against AlphaGo Zero!')
    print('--------------------------------------------------------')
    
    player_start = True if player_start == 'y' else False
    while True:
        if player_start in [True, None]:
            player_start = None
            print(f'\nThe current board looks like this:\n{board}\n')
            valid_moves = get_valid_moves(board)
            reward = end_game(board, valid_moves)
            if reward == -1:
                print('*******')
                print("Oh no, it's looks like you lost!")
                break
            elif reward == 0:
                print('*******')
                print("It's a draw!")
                break

            print('It is your turn.')
            chance_of_win = alpha_go.value_model.predict(np.expand_dims(board, axis=0)).flatten()[0]
            print(f'AlphaGo belives you have a {(chance_of_win+1)/2:.0%} chance of winning from here.\n')
            while True:
                print(f'Pick a valid board position to move: {valid_moves}')
                action = int(input('Your move is: '))
                if action not in valid_moves:
                    print('\nYour action is invalid, please try again')
                else:
                    break
            board[action] = 1
            print('------')

        if player_start in [False, None]:
            player_start = None
            print(f'\nThe current board looks like this:\n{board}\n')
            board = flip_board(board)
            valid_moves = get_valid_moves(board)
            reward = end_game(board, valid_moves)
            if reward == -1:
                print('*******')
                print("You won!")
                break
            elif reward == 0:
                print('*******')
                print("It's a draw!")
                break

            print("AlphaGo Zero's turn")
            chance_of_win = alpha_go.value_model.predict(np.expand_dims(board, axis=0)).flatten()[0]
            print(f'AlphaGo Zero belives it has a {(chance_of_win+1)/2:.0%} chance of winning from here.\n')
            alpha_actions = alpha_go.policy_model.predict(np.expand_dims(board, axis=0)).flatten()
            alpha_actions = clean_nn_probs(board, alpha_actions)
            print(f"This is what is looks like inside AlphaGo Zero's brain:\n{alpha_actions}\n")
            if difficulty == 'easy':
                alpha_action = np.random.choice([0,1,2,3], p=rescale_probs)
            else:
                alpha_action = np.argmax(alpha_actions)
            print(f"AlphaGo Zero's move is: {alpha_action}")
            board[alpha_action] = 1
            board = flip_board(board)
            print('------\n')

    play_again = input('Play again? [y/n]')
    if play_again != 'y':
        break

Please select difficulty [(easy/hard]e
Would you like to start? [(y/n]y
--------------------------------------------------------
Welcome to a new game of connect 2 against AlphaGo Zero!
--------------------------------------------------------

The current board looks like this:
[0 0 0 0]

It is your turn.
AlphaGo belives you have a 70% chance of winning from here.

Pick a valid board position to move: [0 1 2 3]
Your move is: 1
------

The current board looks like this:
[0 1 0 0]

AlphaGo Zero's turn
AlphaGo Zero belives it has a 15% chance of winning from here.

This is what is looks like inside AlphaGo Zero's brain:
[0.2418794  0.         0.36305967 0.3950609 ]

AlphaGo Zero's move is: 3
------


The current board looks like this:
[ 0  1  0 -1]

It is your turn.
AlphaGo belives you have a 98% chance of winning from here.

Pick a valid board position to move: [0 2]
Your move is: 0
------

The current board looks like this:
[ 1  1  0 -1]

*******
You won!
Play again? [y/n]y
Please selec